# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [ ]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [12]:
# imports

import ollama

In [13]:
MODEL = "llama3.2"

In [14]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [16]:
def user_prompt_generator(website) -> str:
    user_prompt = f"You will act as a website summarizer with knowledge of Web Content Accessibility Guidelines. You will look into the web: {website.title} and "
    user_prompt += "break down the relevant information about it in this categories: What is the website about, \
    to whom the website belongs and what practises should improve to have a better user experience. \n\n"
    user_prompt += website.text

    return user_prompt

In [23]:
def messages_generator(user_prompt : str) -> list[dict[str, str]]:
    messages = [{"role" : "user", "content" : user_prompt}]

    return messages

In [21]:
def user_request_reader() -> str:
    while True:
        website_url = input("Define what website you want to summarize by giving the url: ")
        if website_url.lower().startswith("http"):
            return website_url
        print("URL not valid. Please provide a full url.\n")

In [19]:
def summarizer_bot():
    website_url = user_request_reader()
    website = Website(website_url)
    
    user_prompt = user_prompt_generator(website)
    messages = messages_generator(user_prompt)

    response = ollama.chat(model=MODEL, messages=messages)
    print(response['message']['content'])

In [24]:
# The call
summarizer_bot()

Define what website you want to summarize by giving the url:  test.com


URL not valid. Please provide a full url.



Define what website you want to summarize by giving the url:  https://edwarddonner.com


**Summary:**

The website "Home - Edward Donner" belongs to Edward Donner, a co-founder and CTO of Nebula.io, an AI startup. The website is about Edward's interests in writing code, experimenting with Large Language Models (LLMs), and DJing, as well as his work in applying AI to help people discover their potential.

**Categories:**

### What is the website about?

The website is primarily about Edward Donner's personal brand, showcasing his expertise in AI and LLMs. It includes information about his work at Nebula.io, which applies AI to talent management. The website also features a "Connect Four" arena where LLMs compete against each other, as well as sections for learning more about LLMs and staying up-to-date with Edward's courses and publications.

### To whom does the website belong?

The website belongs to Edward Donner, a co-founder and CTO of Nebula.io. It appears to be a personal website or blog, showcasing his expertise and interests in AI and LLMs.

### Practices to improv